In [1]:
!pip install --upgrade langchain openai -q

In [3]:
!pip install sentence_transformers -q

In [4]:
!pip install unstructured -q
!pip install unstructured[local-inference] -q
!pip install detectron2@git+https://github.com/facebookresearch/detectron2.git@v0.6 -q

zsh:1: no matches found: unstructured[local-inference]


In [7]:
pip install unstructured

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [66]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import pdf2image
import re


In [65]:
directory = '../bio_papers/' 

def load_docs(directory):
    loader = DirectoryLoader(directory)
    docs = loader.load()
    return docs

documents = load_docs(directory)
len(documents)

10

In [62]:

def split_docs(documents, chunk_size = 1000, chunk_overlap = 20):
    splitter = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap = chunk_overlap)
    chunks = splitter.split_documents(documents)
    return chunks

chunks = split_docs(documents)
len(chunks)

1124

In [67]:
def load_docs(directory):
    loader = DirectoryLoader(directory)
    docs = loader.load()
    return docs

def get_section_type(section):
    """Determine the type of a section based on its first line."""
    first_line = section.split("\n")[0].lower()
    candidates = [(priority, section_type) for section_type, (priority, keywords) in SECTION_KEYWORDS.items() if any(keyword in first_line for keyword in keywords)]
    # Return the section type with the highest priority, or "OTHER" if there are no candidates
    return max(candidates, default=(0, "OTHER"))[1]

def split_into_sections(document):
    """Split a document into sections at each header."""
    sections = re.split(r"\n(?=.{1," + str(MAX_HEADER_LENGTH) + r"}:|.{1," + str(MAX_HEADER_LENGTH) + r"}\n\n)", document)
    return {get_section_type(section): section for section in sections}

def split_docs(documents, chunk_size=1000, chunk_overlap=20):
    """Split each section of each document into chunks."""
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = []
    for document in documents:
        print(type(document))  # Add this line
        sections = split_into_sections(document)
        for section, text in sections.items():
            chunks.extend(splitter.split_documents(text))
    return chunks 

In [84]:
import PyPDF2
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import re

# Assign a priority to each section type based on its typical order in a research paper
SECTION_KEYWORDS = {
    "INTRODUCTION": (1, ["introduction", "background", "summary"]),
    "METHODS": (2, ["methods", "methodology", "experimental procedure", "approach"]),
    "RESULTS": (3, ["results", "findings", "data"]),
    "DISCUSSION": (4, ["discussion", "interpretation", "analysis"]),
    "CONCLUSION": (5, ["conclusion", "summary", "outcomes"])
}
MAX_HEADER_LENGTH = 50  # Maximum length of a header line

directory = '../bio_papers/'

class Document:
    def __init__(self, page_content, metadata=None):
        self.page_content = page_content
        self.metadata = metadata

def load_docs(directory):
    docs = []
    file_list = os.listdir(directory)
    print(type(file_list))  # Print the type of object returned by os.listdir(directory)
    
    for filename in file_list:
        if filename.endswith(".pdf"):
            with open(os.path.join(directory, filename), 'rb') as file:
                reader = PyPDF2.PdfReader(file)
                print(type(reader))  # Print the type of object returned by PyPDF2.PdfReader(file)
                
                text = ''
                for page in range(reader.pages()):
                    text += reader.getPage(page).extractText()
                docs.append(text)
    return docs

def get_section_type(section):
    """Determine the type of a section based on its first line."""
    first_line = section.split("\n")[0].lower()
    candidates = [(priority, section_type) for section_type, (priority, keywords) in SECTION_KEYWORDS.items() if any(keyword in first_line for keyword in keywords)]
    # Return the section type with the highest priority, or "OTHER" if there are no candidates
    return max(candidates, default=(0, "OTHER"))[1]

def split_into_sections(document):
    """Split a document into sections at each header."""
    sections = re.split(r"\n(?=.{1," + str(MAX_HEADER_LENGTH) + r"}:|.{1," + str(MAX_HEADER_LENGTH) + r"}\n\n)", document)
    return {get_section_type(section): Document(section) for section in sections}

def split_docs(documents, chunk_size=1000, chunk_overlap=20):
    """Split each section of each document into chunks."""
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = []
    for document in documents:
        sections = split_into_sections(document)
        for section, doc in sections.items():
            chunks.extend(splitter.split_documents([doc]))
    return chunks

# Load documents
documents = load_docs(directory)

# Split documents into chunks
chunks = split_docs(documents)

print(f"Loaded {len(documents)} documents")
print(f"Split into {len(chunks)} chunks")


<class 'list'>
<class 'PyPDF2._reader.PdfReader'>


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /var/folders/cm/19r9pjlj5sz023y4mpj99ggh0000gn/T/ipykernel_55933/2276783812.py:63 in <module>    │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/cm/19r9pjlj5sz023y4mpj99ggh0000gn/T/ipykernel_55933/2276783812.py'                 │
│                                                                                                  │
│ /var/folders/cm/19r9pjlj5sz023y4mpj99ggh0000gn/T/ipykernel_55933/2276783812.py:35 in load_docs   │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/var/folders/cm/19r9pjlj5sz023y4mpj99ggh0000gn/T/ipykernel_55933/2276783812.py'                 │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: '_VirtualList' object is not callable

In [77]:
chunks[0].page_content

'I\n\n\n\nN H P A A u t h o r\n\nM a n u s c r i p t\n\nI\n\n\n\nN H P A A u t h o r\n\nM a n u s c r i p t\n\nI\n\n\n\nN H P A A u t h o r\n\nM a n u s c r i p t\n\nNIH Public Access Author Manuscript Cell Metab. Author manuscript; available in PMC 2015 August 05.\n\nPublished in final edited form as:\n\nCell Metab. 2014 August 5; 20(2): 368–375. doi:10.1016/j.cmet.2014.06.003.\n\nBone marrow adipose tissue is an endocrine organ that contributes to increased circulating adiponectin during caloric restriction'

In [58]:
from langchain.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name = 'all-MiniLM-L6-v2')

In [59]:
query_result = embeddings.embed_query('What is the meaning of life?')
len(query_result)

384

In [51]:
!pip install pinecone-client -q

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [60]:
import pinecone 
from langchain.vectorstores import Pinecone 

# initialize pinecone


index_name = 'langchain-chatbot'
index = Pinecone.from_documents(chunks, embeddings, index_name = index_name)

In [61]:
def get_similar_docs(query, k=1, score=False): 
    if score: 
        similar_docs = index.similarity_search_with_scores(query, k=k)
    else:
        similar_docs = index.similarity_search(query, k=k)
    return similar_docs

query = 'how do i introduce a mutation'
similar_docs = get_similar_docs(query)
similar_docs

[Document(page_content='Assessment of mutations', metadata={'source': '../bio_papers/main_1.pdf'})]